# Similitud entre productos

## Descripción
Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen.
Para este desafío tenemos un dataset “items_titles.csv” que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil.

## Entregable
El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset “item_titles_test.csv” generando como output un listado de la
forma donde ordenando por score de similitud podamos encontrar los pares de productos más similares en nuestro dataset de test.
    
Considere el tiempo de ejecución y analice si la solución es escalable, mostrando el tiempo de ejecución en función del tamaño de la lista a procesar.
obs: Para este ejercicio, no utilice modelos previamente entrenados.nados.

## Importando las bibliotecas

In [ ]:
pip install faiss-cpu

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import faiss
import numpy as np
import time

## Importando los datos

In [25]:
df_itens = pd.read_csv("items_titles.csv")

In [29]:
df_itens.shape

(30000, 1)

In [27]:
df_itens

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...
...,...
29995,Tênis Vans Old Skool I Love My Vans - Usado - ...
29996,Tênis Feminino Preto Moleca 5296155
29997,Tenis Botinha Com Pelo Via Marte Original Lanç...
29998,Tênis Slip On Feminino Masculino Original Sapa...


Para determinar la similitud entre los títulos de productos de un conjunto de datos como item_titles_test.csv y generar las parejas más similares basadas en una puntuación de similitud, sin utilizar modelos preentrenados, podemos emplear varios enfoques de similitud de texto. A continuación, te explico cómo abordar el problema paso a paso:

Enfoque clave
Preprocesamiento de texto: Limpiar y preprocesar los títulos de los productos.
Cálculo de similitud: Utilizar medidas de similitud de texto, tales como:
Similitud de Jaccard (basada en la intersección de conjuntos).
Similitud del coseno (basada en vectores de frecuencia de términos).

Preprocesamiento:
La función preprocess_text() convierte los títulos a minúsculas, lo que puede ampliarse para incluir la eliminación de signos de puntuación, eliminación de palabras vacías (stopwords), y técnicas más avanzadas.

Vectorización TF-IDF:
El vectorizador TF-IDF (Frecuencia de Término - Frecuencia Inversa de Documento) transforma los datos de texto en vectores numéricos basados en la frecuencia de las palabras, ajustado por lo común que es una palabra en todos los títulos.

Similitud del coseno:
La función cosine_similarity calcula la similitud por pares entre los vectores de los títulos de los productos. Esta similitud varía entre 0 (sin similitud) y 1 (similitud perfecta).

Extracción de pares:
Dado que la similitud del coseno produce una matriz de similitud de tamaño n x n (donde n es el número de títulos), solo extraemos la parte superior de la matriz (usando itertools.combinations) para evitar comparaciones redundantes (es decir, comparar el título A con el título B y luego el título B con el título A).

Ordenación:
Las puntuaciones de similitud se ordenan en orden descendente, lo que nos permite identificar las parejas más similares.

Escalabilidad y tiempo de ejecución:

La complejidad temporal de este enfoque depende del tamaño del conjunto de datos:

El paso de vectorización (TF-IDF) escala linealmente con el número de títulos de productos.
El cálculo de similitud (Similitud del Coseno) es O(n²), donde n es el número de títulos, debido a la necesidad de comparaciones por pares.
Esto hace que la solución sea adecuada para conjuntos de datos pequeños a medianos. Para conjuntos de datos muy grandes, podrían necesitarse optimizaciones como los métodos de Aproximación de Vecinos Más Cercanos (ANN) para reducir el número de comparaciones.

Tiempo de ejecución:
El tiempo que toma este enfoque dependerá del número de títulos en el conjunto de datos. Para obtener el tiempo de ejecución como función del tamaño del conjunto de datos, se puede ejecutar el código con diferentes tamaños del conjunto de datos (por ejemplo, 100, 1,000, 10,000 títulos) y observar la diferencia de tiempo.

In [ ]:
# Preprocess the data
def preprocess_text(text):
    return text.lower()

df_itens['cleaned_title'] = df_itens['title'].fillna('').apply(preprocess_text)

# Vectorize the titles using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_title'])

# Define a function to calculate similarity in batches
def batch_cosine_similarity(tfidf_matrix, batch_size=1000):
    n = tfidf_matrix.shape[0]
    pairs = []
    similarity_scores = []

    for i in range(0, n, batch_size):
        end = min(i + batch_size, n)
        batch_matrix = tfidf_matrix[i:end]
        batch_similarity = cosine_similarity(batch_matrix, tfidf_matrix)  # Similarity of each batch with the entire set

        for row_idx, row in enumerate(batch_similarity):
            for col_idx, score in enumerate(row):
                if i + row_idx < col_idx:  # Avoid duplicate pairs
                    pairs.append((i + row_idx, col_idx, score))

    return pairs

# Step 3: Calculate similarity in batches
start_time = time.time()
similarity_scores = batch_cosine_similarity(tfidf_matrix, batch_size=1000)
end_time = time.time()

# Step 4: Sort the results by similarity score
similarity_scores = sorted(similarity_scores, key=lambda x: x[2], reverse=True)

# Step 5: Output the top 10 most similar product pairs
top_n = 10
for i, j, score in similarity_scores[:top_n]:
    print(f"Product 1: {df['title'][i]} \nProduct 2: {df['title'][j]} \nSimilarity Score: {score}\n")

# Measure execution time
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

Conclusión
Este enfoque utiliza la vectorización TF-IDF y la similitud del coseno para calcular la similitud entre títulos de productos, lo cual es interpretable y escalable para conjuntos de datos pequeños a medianos. Para conjuntos de datos de gran tamaño, técnicas más avanzadas como el Locality-Sensitive Hashing (LSH) u otros métodos de vecinos más cercanos aproximados pueden optimizar aún más el proceso. También se puede considerar el uso de la distancia de Levenshtein (distancia de edición entre dos cadenas) como otra métrica de similitud eficiente.